
# 🧠 Aurelion — Ejercicios en Jupyter Notebook

Este cuaderno resuelve los **ejercicios de análisis de ventas** usando tu módulo `procesoDatos` y responde a las preguntas P1–P15 descritas en la documentación del proyecto.

> Ejecuta las celdas **en orden**, desde arriba hacia abajo. Si faltan archivos Excel, el cuaderno mostrará mensajes claros para ayudarte a ubicarlos.


In [7]:

# ✅ Dependencias base
import sys, os, importlib.util, textwrap
import pandas as pd
import numpy as np

print(sys.version)
print("Pandas:", pd.__version__)


3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
Pandas: 1.5.3



## 1) Importar el módulo `procesoDatos` del repositorio

Intentamos cargar automáticamente **`procesoDatos - copia.py`** (nombre exacto del archivo que compartiste).  
Si lo renombraste a `procesoDatos.py`, igual funcionará.


In [8]:

# 🔎 Carga robusta del módulo, sin importar el nombre exacto del archivo
from importlib.util import spec_from_file_location, module_from_spec

candidate_paths = [
    'procesoDatos.py',
    'procesoDatos - copia.py',
    './src/procesoDatos.py',
    './notebooks/procesoDatos.py',
]

loaded_module = None
for path in candidate_paths:
    if os.path.exists(path):
        spec = spec_from_file_location("procesoDatos_mod", path)
        mod = module_from_spec(spec)
        spec.loader.exec_module(mod)
        loaded_module = mod
        print(f"✅ Módulo cargado desde: {path}")
        break

if loaded_module is None:
    raise FileNotFoundError("No se encontró 'procesoDatos.py' ni 'procesoDatos - copia.py' en el directorio actual. "
                            "Copia el archivo a esta carpeta o ajusta la lista 'candidate_paths'.")

# Alias convenientes
cargar_datos = loaded_module.cargar_datos
generar_campos_calculados = loaded_module.generar_campos_calculados
crear_df_maestro = loaded_module.crear_df_maestro


✅ Módulo cargado desde: procesoDatos - copia.py



## 2) Cargar datos y construir el **DataFrame maestro**

Esto usará la lógica de búsqueda de rutas definida en tu módulo para encontrar los Excel.


In [9]:

# 📥 Carga de datos y armado del maestro
dfs = cargar_datos()
dfs = generar_campos_calculados(dfs)
df_master = crear_df_maestro(dfs)

# Mostrar vista rápida
print("✅ DataFrames cargados: ", list(dfs.keys()))
display(df_master.head())
print("Filas maestro:", len(df_master))


✅ DataFrames cargados:  ['clientes', 'productos', 'ventas', 'detalle']


,id_venta,id_producto,nombre_producto_detalle,cantidad,precio_unitario_detalle,importe,costo_unitario,ganancia_bruta,fecha_venta,id_cliente,...,email_venta,medio_pago,monto_total,nombre_cliente,email_cliente,ciudad,fecha_registro,nombre_producto_productos,categoria,precio_unitario_productos
0,1,90,Toallas Húmedas x50,1,2902,2902,2232.31,669.69,2024-06-19,62,...,guadalupe.romero@mail.com,tarjeta,2902,Guadalupe Romero,guadalupe.romero@mail.com,Carlos Paz,2023-03-03,Toallas Húmedas x50,Higiene personal,2902
1,2,82,Aceitunas Negras 200g,5,2394,11970,1841.54,2762.30,2024-03-17,49,...,olivia.gomez@mail.com,qr,34186,Olivia Gomez,olivia.gomez@mail.com,Rio Cuarto,2023-02-18,Aceitunas Negras 200g,Alimentos,2394
2,2,39,Helado Vainilla 1L,5,469,2345,360.77,541.15,2024-03-17,49,...,olivia.gomez@mail.com,qr,34186,Olivia Gomez,olivia.gomez@mail.com,Rio Cuarto,2023-02-18,Helado Vainilla 1L,Alimentos,469
3,2,70,Fernet 750ml,2,4061,8122,3123.85,1874.30,2024-03-17,49,...,olivia.gomez@mail.com,qr,34186,Olivia Gomez,olivia.gomez@mail.com,Rio Cuarto,2023-02-18,Fernet 750ml,Alimentos,4061
4,2,22,Medialunas de Manteca,1,2069,2069,1591.54,477.46,2024-03-17,49,...,olivia.gomez@mail.com,qr,34186,Olivia Gomez,olivia.gomez@mail.com,Rio Cuarto,2023-02-18,Medialunas de Manteca,Alimentos,2069


Filas maestro: 343



## 3) Funciones de ayuda para los ejercicios


In [10]:

def pareto_clientes(df_ventas, top_pct=0.80):
    """Identifica el conjunto de clientes que acumula ~80% de los ingresos."""
    base = df_ventas.groupby('id_cliente', as_index=False)['monto_total'].sum()
    base = base.sort_values('monto_total', ascending=False)
    base['pct_ingresos'] = base['monto_total'] / base['monto_total'].sum()
    base['pct_acumulado'] = base['pct_ingresos'].cumsum()
    base['pareto_80'] = base['pct_acumulado'] <= top_pct
    return base

def primeras_compras(df_ventas):
    """Para cada cliente, devuelve el id_venta & fecha de su primera compra."""
    tmp = df_ventas.sort_values(['id_cliente', 'fecha_venta'])
    first = tmp.groupby('id_cliente', as_index=False).first()[['id_cliente', 'id_venta', 'fecha_venta']]
    first.rename(columns={'id_venta':'id_venta_primera'}, inplace=True)
    return first

def agregar_ciudad(df_master):
    # df_master ya incluye ciudad desde clientes; asegurar presencia
    if 'ciudad' not in df_master.columns:
        raise ValueError("No se encuentra la columna 'ciudad' en df_master. Revisa el join de clientes.")
    return df_master

def trimestre(dt):
    return f"Q{((dt.month-1)//3)+1}-{dt.year}"



## 4) Análisis de Clientes (P1–P4)


In [12]:
# Asegurar ventas con 'monto_total'
ventas = dfs['ventas'].copy()

# P1 — Clientes que generan ~80% de ingresos (Pareto)
pareto = pareto_clientes(ventas, top_pct=0.80)
top_pareto = pareto[pareto['pareto_80']]
display(pareto.head(10))
print(f"Clientes en el 80%% de ingresos: {top_pareto.shape[0]} / {pareto.shape[0]}")

# P2 — Promedio, mínimo y máximo por cliente
p2 = ventas.groupby('id_cliente')['monto_total'].agg(['mean','min','max']).reset_index()
p2.columns = ['id_cliente','promedio_compra','min_compra','max_compra']
display(p2.head(10))

# P3 — Frecuencia de compra y productos por clientes más fieles
# Definimos "clientes fieles" como top 20% por número de compras
freq = ventas.groupby('id_cliente', as_index=False)['id_venta'].count().rename(columns={'id_venta':'n_compras'})
cut = int(max(1, np.ceil(0.2 * len(freq))))
top_frecuencia = freq.sort_values('n_compras', ascending=False).head(cut)

# 🔧 Productos más frecuentes de estos clientes (fix: añadir id_cliente a 'detalle' desde 'ventas')
detalle = dfs['detalle'].copy()
ventas_min = dfs['ventas'][['id_venta', 'id_cliente']].drop_duplicates()
detalle_cli = detalle.merge(ventas_min, on='id_venta', how='left')

prod_fieles = (
    detalle_cli.merge(top_frecuencia[['id_cliente']], on='id_cliente', how='inner')
               .groupby('id_producto', as_index=False)['cantidad'].sum()
               .sort_values('cantidad', ascending=False)
)

display(top_frecuencia.head())
display(prod_fieles.head(10))

# P4 — Cliente que más compra (por monto)
top_cliente = (
    ventas.groupby('id_cliente', as_index=False)['monto_total'].sum()
          .sort_values('monto_total', ascending=False)
          .head(1)
)
display(top_cliente)


,id_cliente,monto_total,pct_ingresos,pct_acumulado,pareto_80
3,5,132158,0.049844,0.049844,True
41,56,90701,0.034209,0.084053,True
38,52,90522,0.034141,0.118194,True
19,25,81830,0.030863,0.149057,True
0,1,72448,0.027324,0.176381,True
36,49,71321,0.026899,0.203280,True
44,61,67959,0.025631,0.228911,True
57,84,67575,0.025486,0.254398,True
50,72,65001,0.024516,0.278913,True
29,39,64786,0.024434,0.303348,True


Clientes en el 80%% de ingresos: 39 / 67


,id_cliente,promedio_compra,min_compra,max_compra
0,1,36224.0,36035,36413
1,2,22150.0,22150,22150
2,3,33310.0,33310,33310
3,5,33039.5,11832,45142
4,6,24439.0,11622,37256
5,8,61503.0,61503,61503
6,9,20920.0,11608,30232
7,10,25860.0,25860,25860
8,12,24759.0,15368,34150
9,13,13188.0,13188,13188


,id_cliente,n_compras
41,56,5
3,5,4
32,42,4
36,49,4
50,72,4


,id_producto,cantidad
26,43,16
48,79,15
8,18,12
18,32,12
54,86,12
37,59,11
23,39,9
50,81,9
47,76,8
52,83,7


,id_cliente,monto_total
3,5,132158



## 5) Análisis de Productos (P5–P7)


In [14]:
# ====== Celda robusta para P5–P7 ======

master = df_master.copy()

# Normalizar nombres de columnas a minúsculas (evita sorpresas)
master.columns = [c.strip().lower() for c in master.columns]
if 'productos' in dfs:
    df_prod = dfs['productos'].copy()
    df_prod.columns = [c.strip().lower() for c in df_prod.columns]
else:
    df_prod = None

# --- Asegurar columnas clave en master ---

# 1) nombre_producto
if 'nombre_producto' not in master.columns:
    if df_prod is not None and 'id_producto' in df_prod.columns:
        # posibles nombres de "nombre de producto" en la tabla productos
        name_cands = ['nombre_producto','nombre','producto','descripcion','product_name','name']
        name_col = next((c for c in name_cands if c in df_prod.columns), None)

        # posibles nombres de "categoria" en la tabla productos
        cat_cands = ['categoria','category','categoria_producto','categoria_prod']
        cat_col = next((c for c in cat_cands if c in df_prod.columns), None)

        cols = ['id_producto']
        if name_col: cols.append(name_col)
        if cat_col:  cols.append(cat_col)

        if len(cols) > 1:
            prod_rename = df_prod[cols].rename(columns={
                (name_col or 'nombre_producto'): 'nombre_producto',
                (cat_col  or 'categoria'): 'categoria'
            })
            master = master.merge(prod_rename, on='id_producto', how='left')
        else:
            # no se encontró columna de nombre en productos; seguiremos sin nombre
            pass

# 2) categoria (por si no venía en master)
if 'categoria' not in master.columns and df_prod is not None and 'id_producto' in df_prod.columns:
    cat_cands = ['categoria','category','categoria_producto','categoria_prod']
    cat_col = next((c for c in cat_cands if c in df_prod.columns), None)
    if cat_col:
        master = master.merge(
            df_prod[['id_producto', cat_col]].rename(columns={cat_col: 'categoria'}),
            on='id_producto', how='left'
        )

# --- P5: Categoría con mayor cantidad vendida e ingresos ---
# Si aún no existe 'categoria', marcamos 'Sin categoría' para no romper
if 'categoria' not in master.columns:
    master['categoria'] = 'Sin categoría'

# Asegurar métricas presentes
for needed in ['cantidad', 'importe']:
    if needed not in master.columns:
        raise KeyError(f"Falta la columna '{needed}' en df_master. Revisa crear_df_maestro().")

p5 = (master.groupby('categoria', as_index=False)
            .agg(cantidad_total=('cantidad', 'sum'),
                 ingresos_total=('importe', 'sum'))
            .sort_values('cantidad_total', ascending=False))
display(p5)

# --- P6: Top 10 productos menos vendidos ---
# Si no tenemos 'nombre_producto', usamos solo id_producto y luego (opcional) volvemos a unir nombre si existe
if 'nombre_producto' in master.columns:
    grp_cols = ['id_producto', 'nombre_producto']
else:
    grp_cols = ['id_producto']

p6 = (master.groupby(grp_cols, as_index=False)['cantidad']
           .sum()
           .sort_values('cantidad', ascending=True)
           .head(10))

# Si no había nombre y lo podemos añadir ahora desde productos, lo hacemos para mostrar más lindo
if 'nombre_producto' not in p6.columns and df_prod is not None:
    name_cands = ['nombre_producto','nombre','producto','descripcion','product_name','name']
    name_col = next((c for c in name_cands if c in df_prod.columns), None)
    if name_col:
        p6 = p6.merge(
            df_prod[['id_producto', name_col]].rename(columns={name_col: 'nombre_producto'}),
            on='id_producto', how='left'
        )

display(p6)

# --- P7: Productos más frecuentes en primeras compras ---
first = primeras_compras(dfs['ventas'])

p7_base = first.merge(master, left_on='id_venta_primera', right_on='id_venta', how='left')

if 'nombre_producto' in p7_base.columns:
    grp_cols_p7 = ['id_producto', 'nombre_producto']
else:
    grp_cols_p7 = ['id_producto']

p7 = (p7_base.groupby(grp_cols_p7, as_index=False)['cantidad']
              .sum()
              .sort_values('cantidad', ascending=False)
              .head(10))

# Añadir nombre si faltaba y está en productos
if 'nombre_producto' not in p7.columns and df_prod is not None:
    name_cands = ['nombre_producto','nombre','producto','descripcion','product_name','name']
    name_col = next((c for c in name_cands if c in df_prod.columns), None)
    if name_col:
        p7 = p7.merge(
            df_prod[['id_producto', name_col]].rename(columns={name_col: 'nombre_producto'}),
            on='id_producto', how='left'
        )

display(p7)



,categoria,cantidad_total,ingresos_total
0,Alimentos,846,2214681
1,Higiene personal,93,238689
2,Limpieza,77,198047


,id_producto,nombre_producto,cantidad
51,52,Detergente Líquido 750ml,2
32,33,Chocolate con Leche 100g,2
25,26,Alfajor Triple,2
24,25,Galletitas Vainilla,2
47,48,Porotos Negros 500g,3
59,61,Miel Pura 250g,3
29,30,Maní Salado 200g,3
15,16,Yogur Natural 200g,3
62,64,Avena Instantánea 250g,4
65,67,Vino Tinto Malbec 750ml,4


,id_producto,nombre_producto,cantidad
45,53,Lavandina 1L,18
35,43,Salsa de Tomate 500g,18
31,38,Mermelada de Frutilla 400g,18
33,41,Aceite de Girasol 1L,17
62,76,Pizza Congelada Muzzarella,16
67,81,Aceitunas Verdes 200g,15
59,72,Ron 700ml,15
6,7,Jugo de Manzana 1L,14
69,83,Queso Untable 190g,13
82,98,Desengrasante 500ml,13



## 6) Geografía y Medios de Pago (P8–P10)


In [15]:

master = agregar_ciudad(df_master)

# P8 — Distribución geográfica de ingresos
p8 = master.groupby('ciudad', as_index=False)['importe'].sum()
p8['pct'] = p8['importe'] / p8['importe'].sum()
display(p8.sort_values('importe', ascending=False))

# P9 — Volumen promedio en primeros 30 días por ciudad
clientes = dfs['clientes'].copy()
ventas = dfs['ventas'].copy()
ventas_30 = ventas.merge(clientes[['id_cliente','fecha_registro','ciudad']], on='id_cliente', how='left')
ventas_30['dias_desde_registro'] = (ventas_30['fecha_venta'] - ventas_30['fecha_registro']).dt.days
ventas_30win = ventas_30[ventas_30['dias_desde_registro'].between(0, 30, inclusive='both')]

# monto_total ya viene agregado por venta
p9 = ventas_30win.groupby('ciudad', as_index=False)['monto_total'].mean().rename(columns={'monto_total':'promedio_30d'})
display(p9.sort_values('promedio_30d', ascending=False))

# P10 — Porcentaje de ventas por medio de pago (global y por ciudad)
por_medio = ventas.groupby('medio_pago', as_index=False)['monto_total'].sum()
por_medio['pct'] = por_medio['monto_total'] / por_medio['monto_total'].sum()
display(por_medio.sort_values('pct', ascending=False))

por_medio_ciudad = ventas.merge(clientes[['id_cliente','ciudad']], on='id_cliente', how='left')                          .groupby(['ciudad','medio_pago'], as_index=False)['monto_total'].sum()
por_medio_ciudad['pct_ciudad'] = por_medio_ciudad.groupby('ciudad')['monto_total'].transform(lambda s: s/s.sum())
display(por_medio_ciudad.sort_values(['ciudad','pct_ciudad'], ascending=[True, False]).head(20))


,ciudad,importe,pct
4,Rio Cuarto,792203,0.298785
0,Alta Gracia,481504,0.181603
2,Cordoba,481482,0.181594
1,Carlos Paz,353852,0.133458
5,Villa Maria,313350,0.118182
3,Mendiolaza,229026,0.086379


,ciudad,promedio_30d


,medio_pago,monto_total,pct
0,efectivo,934819,0.352573
1,qr,714280,0.269396
3,transferencia,542219,0.204502
2,tarjeta,460099,0.173529


,ciudad,medio_pago,monto_total,pct_ciudad
1,Alta Gracia,qr,215463,0.447479
0,Alta Gracia,efectivo,190731,0.396115
3,Alta Gracia,transferencia,55387,0.115029
2,Alta Gracia,tarjeta,19923,0.041377
7,Carlos Paz,transferencia,172594,0.487758
4,Carlos Paz,efectivo,81264,0.229655
6,Carlos Paz,tarjeta,66750,0.188638
5,Carlos Paz,qr,33244,0.093949
8,Cordoba,efectivo,238734,0.495832
10,Cordoba,tarjeta,100359,0.208438



## 7) Temporal & Pricing (P12, P13, P14, P15)


In [17]:
# === P14 & P15 (robusto si falta 'precio_unitario' o 'categoria') ===
master = df_master.copy()
master.columns = [c.strip().lower() for c in master.columns]

# Traer productos si existen
df_prod = dfs.get('productos')
if df_prod is not None:
    df_prod = df_prod.copy()
    df_prod.columns = [c.strip().lower() for c in df_prod.columns]

# Asegurar 'categoria'
if 'categoria' not in master.columns and df_prod is not None and 'id_producto' in master.columns and 'id_producto' in df_prod.columns:
    cat_cands = ['categoria','category','categoria_producto','categoria_prod']
    cat_col = next((c for c in cat_cands if c in df_prod.columns), None)
    if cat_col:
        master = master.merge(
            df_prod[['id_producto', cat_col]].rename(columns={cat_col: 'categoria'}),
            on='id_producto', how='left'
        )

if 'categoria' not in master.columns:
    master['categoria'] = 'Sin categoría'

# Asegurar 'precio_unitario'
if 'precio_unitario' not in master.columns:
    # 1) Buscar en master con nombres alternos
    price_cands_master = ['precio','price','unit_price','precio_unit']
    alt_price = next((c for c in price_cands_master if c in master.columns), None)

    if alt_price is not None:
        master['precio_unitario'] = master[alt_price]
    else:
        # 2) Buscar en productos y unir por id_producto
        if df_prod is not None and 'id_producto' in df_prod.columns:
            price_cands_prod = ['precio_unitario','precio','price','unit_price','precio_unit']
            price_col = next((c for c in price_cands_prod if c in df_prod.columns), None)
            if price_col is not None:
                master = master.merge(
                    df_prod[['id_producto', price_col]].rename(columns={price_col: 'precio_unitario'}),
                    on='id_producto', how='left'
                )

# 3) Si aún falta, calcular como importe/cantidad (con guardas)
if 'precio_unitario' not in master.columns:
    import numpy as np
    if 'importe' not in master.columns or 'cantidad' not in master.columns:
        raise KeyError("Faltan columnas 'importe' o 'cantidad' para calcular precio_unitario.")
    master['precio_unitario'] = np.where(master['cantidad'] > 0, master['importe'] / master['cantidad'], np.nan)

# --- P14: Precio unitario promedio por categoría
p14 = (master.groupby('categoria', as_index=False)['precio_unitario']
              .mean()
              .rename(columns={'precio_unitario': 'precio_promedio'}))
display(p14.sort_values('precio_promedio', ascending=False))

# --- P15: Monto promedio por pedido vs precio unitario promedio por categoría
pedido_cat = (master.groupby(['categoria','id_venta'], as_index=False)['importe'].sum()
                     .groupby('categoria', as_index=False)['importe'].mean()
                     .rename(columns={'importe': 'monto_promedio_pedido'}))

p15 = pedido_cat.merge(p14, on='categoria', how='left')
display(p15.sort_values('monto_promedio_pedido', ascending=False))



,categoria,precio_promedio
2,Limpieza,2696.888889
0,Alimentos,2653.118881
1,Higiene personal,2629.466667


,categoria,monto_promedio_pedido,precio_promedio
0,Alimentos,19258.095652,2653.118881
1,Higiene personal,9180.346154,2629.466667
2,Limpieza,7921.880000,2696.888889



## 8) Exportar resultados clave (CSV)

Se generan CSVs en la carpeta del cuaderno para facilitar su uso fuera de Jupyter.


In [ ]:
# Crear carpeta de exportación si no existe
export_dir = "resultados"
os.makedirs(export_dir, exist_ok=True)

export_items = {
    "p1_clientes_pareto.csv": pareto,
    "p2_stats_por_cliente.csv": p2,
    "p5_categorias.csv": p5,
    "p6_productos_menos_vendidos.csv": p6,
    "p7_productos_primeras_compras.csv": p7,
    "p8_ingresos_ciudad.csv": p8,
    "p9_promedio_30d_ciudad.csv": p9,
    "p10_medio_pago_global.csv": por_medio,
    "p10_medio_pago_por_ciudad.csv": por_medio_ciudad,
    "p12_mes.csv": p12_mes,
    "p12_trimestre.csv": p12_tri,
    "p13_activacion.csv": p13,
    "p14_precio_prom_cat.csv": p14,
    "p15_pedido_vs_precio_cat.csv": p15,
}

# Guardar cada archivo dentro de la carpeta
for fname, df in export_items.items():
    export_path = os.path.join(export_dir, fname)
    # Sobrescribe el archivo si ya existe
    df.to_csv(export_path, index=False)
    print(f"✅ Exportado: {export_path}")



✅ Exportado: resultados\clientes_pareto.csv
✅ Exportado: resultados\p2_stats_por_cliente.csv
✅ Exportado: resultados\p5_categorias.csv
✅ Exportado: resultados\p6_productos_menos_vendidos.csv
✅ Exportado: resultados\p7_productos_primeras_compras.csv
✅ Exportado: resultados\p8_ingresos_ciudad.csv
✅ Exportado: resultados\p9_promedio_30d_ciudad.csv
✅ Exportado: resultados\p10_medio_pago_global.csv
✅ Exportado: resultados\p10_medio_pago_por_ciudad.csv
✅ Exportado: resultados\p12_mes.csv
✅ Exportado: resultados\p12_trimestre.csv
✅ Exportado: resultados\p13_activacion.csv
✅ Exportado: resultados\p14_precio_prom_cat.csv
✅ Exportado: resultados\p15_pedido_vs_precio_cat.csv
